In [56]:
import os
import re
import pandas as pd
from sklearn.metrics import cohen_kappa_score as cks

### Building the dataframe with the results

In [49]:
label_files = sorted(os.listdir('../data/img_labeling/'))
label_files

['1st_task_labeling_Amelie_08.09.csv',
 '1st_task_labeling_Gerda_07.09.csv',
 '1st_task_labeling_Marcos_24.08.csv',
 '1st_task_labeling_Renato_31.08.csv',
 '1st_task_labeling_Yalemisew_07.09.csv',
 '2nd_task_labeling_Amelie_08.09.csv',
 '2nd_task_labeling_Gerda_07.09.csv',
 '2nd_task_labeling_Marcos_28.08.csv',
 '2nd_task_labeling_Renato_07.09.csv',
 '2nd_task_labeling_Yalemisew_07.09.csv',
 '3rd_task_labeling_Amelie_08.09.csv',
 '3rd_task_labeling_Gerda_07.09.csv',
 '3rd_task_labeling_Marcos_28.08.csv',
 '3rd_task_labeling_Renato_07.09.csv',
 '3rd_task_labeling_Yalemisew_08.09.csv']

In [53]:
list_dfs = []
for label_file in label_files:
    task = int(re.findall(r'[1-3]{1}', label_file)[0])
    user = re.findall(r'Amelie|Gerda|Renato|Yalemisew|Marcos', label_file)[0]
    date = re.findall(r'[0-9]{2}.[0-9]{2}', label_file)[0]
    date = f'{date}.2020'
    df = pd.read_csv(f'../data/img_labeling/{label_file}', names=['class','image name', 'w','h'])
    df['task'] = task
    df['user'] = user
    df['date'] = date
    df['date'] = pd.to_datetime(df['date'])
    df.drop(['w','h'], axis=1, inplace=True)
    df = df[['task', 'user', 'date', 'image name', 'class']]
    list_dfs.append(df)

In [54]:
df_labeling = pd.concat(list_dfs)
df_labeling['id_image'] = pd.Categorical(df_labeling['image name']).codes
df_labeling = df_labeling[['task', 'user', 'date', 'image name', 'id_image', 'class']]
df_labeling.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6232 entries, 0 to 451
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   task        6232 non-null   int64         
 1   user        6232 non-null   object        
 2   date        6232 non-null   datetime64[ns]
 3   image name  6232 non-null   object        
 4   id_image    6232 non-null   int16         
 5   class       6232 non-null   object        
dtypes: datetime64[ns](1), int16(1), int64(1), object(3)
memory usage: 304.3+ KB


In [55]:
df_labeling.head()

,task,user,date,image name,id_image,class
0,1,Amelie,2020-08-09,203-resource_document_zuiderzeemuseum_B001601_...,88,Non-Fruits
1,1,Amelie,2020-08-09,285-gam19649_1.jpeg,235,Fruits
2,1,Amelie,2020-08-09,07101-O_389_1.jpeg,6,Fruits
3,1,Amelie,2020-08-09,07101-O_927_1.jpeg,12,Fruits
4,1,Amelie,2020-08-09,07101-O_957_1.jpeg,13,Non-Fruits


#### Checking the numbers of images classified:

In [73]:
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        size = len(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image'].unique())
        print(f'Task: {task}, User: {user}, Images Classified: {size}')
    print()

Task: 1, User: Amelie, Images Classified: 392
Task: 1, User: Gerda, Images Classified: 392
Task: 1, User: Marcos, Images Classified: 390
Task: 1, User: Renato, Images Classified: 392
Task: 1, User: Yalemisew, Images Classified: 391

Task: 2, User: Amelie, Images Classified: 392
Task: 2, User: Gerda, Images Classified: 392
Task: 2, User: Marcos, Images Classified: 390
Task: 2, User: Renato, Images Classified: 391
Task: 2, User: Yalemisew, Images Classified: 392

Task: 3, User: Amelie, Images Classified: 392
Task: 3, User: Gerda, Images Classified: 392
Task: 3, User: Marcos, Images Classified: 391
Task: 3, User: Renato, Images Classified: 392
Task: 3, User: Yalemisew, Images Classified: 392



#### Checking which images were not classified:

In [80]:
images = set(df_labeling.id_image.unique())
for task in df_labeling.task.unique():
    for user in df_labeling.user.unique():
        print(f'Task: {task}, User: {user}')
        ids = images.difference(set(df_labeling.loc[(df_labeling.user == user) & (df_labeling.task == task), 'id_image']))
        for id in ids:
            print(df_labeling.loc[(df_labeling.task == task) & (df_labeling.id_image == id), 'image name'].values)
    print()

Task: 1, User: Amelie
['08806-~1.JPE']
['204804~1.JPE']
['204804~2.JPE']
Task: 1, User: Gerda
['2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_1C82EC077B694E00B35B9B8044A16EAB_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_1C82EC077B694E00B35B9B8044A16EAB_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_1C82EC077B694E00B35B9B8044A16EAB_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_1C82EC077B694E00B35B9B8044A16EAB_1.jpeg']
['2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_7000A7800599458180CD0055DB99645F_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_7000A7800599458180CD0055DB99645F_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_7000A7800599458180CD0055DB99645F_1.jpeg'
 '2048047-Athena_Plus_ProvidedCHO_Institutul_Na_ional_al_Patrimoniului_7000A7800599458180CD0055DB99645F_1.jpeg']
['08806-

### Analysing the results

In [65]:
cks(df_labeling.loc[(df_labeling.user == 'Gerda') & (df_labeling.task == 3), 'class'], df_labeling.loc[(df_labeling.user == 'Renato') & (df_labeling.task == 3), 'class'])

ValueError: Found input variables with inconsistent numbers of samples: [452, 392]